# Merge in crosswalks and do aggregations

In [ ]:
import numpy as np
import pandas as pd
import intake
import os

In [ ]:
catalog = intake.open_catalog('../catalogs/*.yml')

In [ ]:
census = pd.read_parquet('s3://hcid-cdbg-project-ita-data/data/raw/raw_census_cleaned.parquet')
census = census[(census.year>=2016) & ((census.table=='pop') | (census.table=='housing'))]

In [ ]:
census = pd.read_parquet('s3://hcid-cdbg-project-ita-data/data/raw/raw_census_cleaned.parquet')

In [ ]:
council_districts = catalog.crosswalk_tracts_council_districts.read()
neighborhood_councils = catalog.crosswalk_tracts_neighborhood_councils.read()
zipcodes = catalog.crosswalk_tracts_zipcodes.read()
congressional_districts = catalog.crosswalk_tracts_congressional_districts.read()
neighborhoods = catalog.crosswalk_tracts_neighborhoods.read()

In [ ]:
boundaries = {'council_districts': council_districts, 'neighborhood_councils': neighborhood_councils,
             'zipcodes': zipcodes, 'congressional_districts': congressional_districts, 'neighborhoods': neighborhoods}

merged_dfs = {}

for key, value in boundaries.items():
    m1 = pd.merge(census, value, on = 'GEOID', how = 'left', validate = 'm:1')
    m1.max_val = m1.max_val.fillna(0)
    n = m1.max_val.max().astype(int)
    for i in range(1, n + 1):
        new_col = f"num{i}"
        allocate_col = f"allocate{i}"
        m1[new_col] = m1.num * m1[allocate_col]
    merged_dfs[key] = m1 

In [ ]:
for key, value in merged_dfs.items():
    display(key)
    display(value.head())